In [1]:

import requests

from dataclasses import dataclass

import json

In [2]:
url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=e16d4ab3-842a-4f39-9ad7-ce5921002280&limit=1000000'  
fileobj = requests.get(url)


In [4]:
jayson = fileobj.json()
json_li = jayson['result']['records']


In [5]:
result_string = json.dumps(json_li)
with open('json_data.json', 'w') as outfile:
    outfile.write(result_string)

In [31]:
Dog_list = []
for dict in json_li:
    li = list(dict.values())
    Dog_list.append(Dog(*li))

In [36]:
Dog_list[0].zip

'15046'

In [32]:
city_limits = [i for i in Dog_list if i.zip in neighborhood_dict.keys()]

{'Allentown/Arlington',
 'Bloomfield',
 'Downtown/Strip District',
 'Highland Park',
 'Lawrenceville',
 'Mount Washington',
 'Northside East',
 'Northside West',
 'Oakland',
 'Perry',
 'Shadyside',
 'Southside Flats',
 'Squirrel Hill',
 'The Hill'}

In [42]:
squirrel_hill = [i for i in city_limits if i.neighborhood == 'Squirrel Hill']

In [43]:
print(len(squirrel_hill))

24


In [44]:
print([i.breed for i in squirrel_hill])

['GER SHEPHERD', 'AM PITT BULL MIX', 'POODLE STANDARD', 'SCHNOODLE', 'YORKSHIRE TERRIER', 'CAV KING CHAR SPANI', 'TERRIER', 'GOLDENDOODLE', 'SFT COAT WHEAT TERR', 'TAG', 'PARSON RUSSELL TERR', 'BRITTANY SPANIEL', 'LAB MIX', 'MIXED', 'STAFFORDSHIRE TERRI', 'SHIBA INU', 'COTON DE TULEAR', 'COTON DE TULEAR', 'MIXED', 'LABRADOR RETRIEVER', 'SHEPHERD MIX', 'MIXED', 'POMERANIAN', 'POMERANIAN']
